In [1]:
import pandas
import konlpy
import gensim

print(pandas.__version__)
print(konlpy.__version__)
print(gensim.__version__)

1.3.3
0.5.2
4.1.2


In [3]:
# 데이터 로드

import pandas as pd

train_data = pd.read_table('~/aiffel/sentiment_classification/data/ratings_train.txt')
test_data = pd.read_table('~/aiffel/sentiment_classification/data/ratings_test.txt')

print(train_data.shape)
train_data.head()

(150000, 3)


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [16]:
# 데이터 로드 구성

from konlpy.tag import Mecab
import numpy as np
from collections import Counter

tokenizer = Mecab()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
stopwords += [",", ".", "'", "\""]
# def load_data(train_data, test_data, num_words=num_words):
#     # 데이터 중복 제거
#     # 결측치 제거
#     # 한국어 토크나이저
#     # 불용어 제거
#     # word_to_index
#     # index_to_word
    
# X_train, y_train, X_test, y_test, word_to_index = load_data(train_data, test_data)

In [5]:
# 중복 제거
_train_data = train_data.drop_duplicates(subset="document")
print(_train_data.shape)

(146183, 3)


In [9]:
# 결측치 제거
_train_data = _train_data.dropna(axis="rows")
print(_train_data.shape)

(146182, 3)


In [37]:
# 토크나이징
okt = konlpy.tag.Okt()
text = "병원 병원은 멀어요. 그 그 그 "
morphs = okt.morphs(text)
print(morphs)

['병원', '병원', '은', '멀어요', '.', '그', '그', '그']


In [38]:
# 불용어 제거

removed_stopwords = [ word for word in morphs if word not in stopwords ]

print(removed_stopwords)

['병원', '병원', '멀어요', '그', '그', '그']


In [44]:
# word_to_index
sentence = remove_stopwords(morphs)

words_limit = 2 #10_000
word_to_index = {}

for sentence in [sentence]:
    for word in sentence:
        if word_to_index.get(word) is not None:
            word_to_index[word] += 1
        else:
            word_to_index[word] = 0

word_to_index = sorted(word_to_index.items(), key=lambda item: item[1], reverse=True)
print(word_to_index)
word_to_index = dict(word_to_index[:words_limit])
print(word_to_index)

[('그', 2), ('병원', 1), ('멀어요', 0)]
{'그': 2, '병원': 1}


## 회고
* 배운 점
  - 전체적인 처리 과정을 상세하게 학습할 수 있었다.
* 아쉬운 점
  - 하니씩 자세하게 보다보니 학습 시간내에는 프로젝트를 다 마무리 못해서 아쉬웠다. 
* 느낀 점
  - 분류 이외 여러 태스크들을 진행해봐야겠다.
* 어려웠던 점
  - 각 내용의 상세한 구현 및 구조를 깊게 파악하는게 쉽지 않은 것 같다.